In [ ]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()
mote2general = station.load_mote2_general()

2025-09-07 15:04:22,821 - servers.jupyter_qcodes.tests.data_loader - INFO - Loading data from /Users/caijiaqi/Documents/GitHub/instrMCP/servers/jupyter_qcodes/tests/data_file/MoTe2QAHE/processed_data.mat
2025-09-07 15:04:22,846 - servers.jupyter_qcodes.tests.data_loader - INFO - Data validation passed: Vtg(220), Vbg(900), lockin arrays(900x220)
2025-09-07 15:04:22,846 - servers.jupyter_qcodes.tests.data_loader - INFO - Data loaded and validated successfully
2025-09-07 15:04:22,847 - servers.jupyter_qcodes.tests.test_instr.general_test_instrument - INFO - Data loaded from /Users/caijiaqi/Documents/GitHub/instrMCP/servers/jupyter_qcodes/tests/data_file/MoTe2QAHE/processed_data.mat
2025-09-07 15:04:22,848 - servers.jupyter_qcodes.tests.test_instr.general_test_instrument - INFO - General test instrument 'mote2_general' initialized from ${instrMCP_PATH}/servers/jupyter_qcodes/tests/data_file/MoTe2QAHE/MoTe2Para.json (expanded: /Users/caijiaqi/Documents/GitHub/instrMCP/servers/jupyter_qcodes

In [6]:
%load_ext instrmcp.servers.jupyter_qcodes.jupyter_mcp_extension

2025-09-07 19:59:33,132 - instrmcp.tools.context7_client - WARNING - Context7 API key not provided - Context7 features disabled
2025-09-07 19:59:33,143 - instrmcp.servers.jupyter_qcodes.jupyter_mcp_extension - INFO - Loading Jupyter QCoDeS MCP extension...
2025-09-07 19:59:33,143 - instrmcp.servers.jupyter_qcodes.active_cell_bridge - INFO - Registered comm target 'mcp:active_cell'
2025-09-07 19:59:33,143 - instrmcp.servers.jupyter_qcodes.jupyter_mcp_extension - INFO - Jupyter QCoDeS MCP extension loaded successfully


✅ QCoDeS MCP extension loaded
🛡️  Default mode: safe
📋 Use %mcp_status to check server status
⚠️  Use %mcp_unsafe to switch to unsafe mode (if needed)
🚀 Use %mcp_start to start the server


In [ ]:
%mcp_status

In [8]:
%mcp_unsafe

⚠️  Mode set to unsafe
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
✅ Mode will take effect when server starts


In [9]:
%mcp_start

2025-09-07 20:00:01,436 - instrmcp.servers.jupyter_qcodes.tools - INFO - QCoDesReadOnlyTools initialized
2025-09-07 20:00:01,448 - instrmcp.servers.jupyter_qcodes.mcp_server - INFO - Jupyter MCP Server initialized on 127.0.0.1:8123 in unsafe mode
2025-09-07 20:00:01,448 - instrmcp.servers.jupyter_qcodes.mcp_server - INFO - Starting Jupyter MCP server on 127.0.0.1:8123


🚀 Starting MCP server...
🚀 QCoDeS MCP Server running on http://127.0.0.1:8123
🔑 Access token: gr8q2zuei_w1z2FRYZOlqMdLEQTIRN94LozUfeZxVjQ


[09/07/25 20:00:01] INFO     Starting MCP server 'Jupyter QCoDeS MCP Server (Unsafe Mode)' with      ]8;id=829873;file:///Users/caijiaqi/miniforge3/lib/python3.10/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=111791;file:///Users/caijiaqi/miniforge3/lib/python3.10/site-packages/fastmcp/server/server.py#1570\1570]8;;\
                             transport 'http' on http://127.0.0.1:8123/mcp                                         

/Users/caijiaqi/miniforge3/lib/python3.10/site-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/Users/caijiaqi/miniforge3/lib/python3.10/site-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [97134]
INFO:     Waiting for application startup.
2025-09-07 20:00:01,481 - mcp.server.streamable_http_manager - INFO - StreamableHTTP session manager started
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8123 (Press CTRL+C to quit)


✅ MCP server started in ⚠️ unsafe mode
⚠️  UNSAFE MODE: execute_editing_cell tool is available


...

2025-09-07 20:00:28,905 - mcp.server.lowlevel.server - INFO - Processing request of type ListToolsRequest


INFO:     127.0.0.1:54234 - "POST /mcp HTTP/1.1" 200 OK


2025-09-07 20:00:35,766 - mcp.server.lowlevel.server - INFO - Processing request of type CallToolRequest


INFO:     127.0.0.1:54259 - "POST /mcp HTTP/1.1" 200 OK


2025-09-07 20:00:49,581 - mcp.server.lowlevel.server - INFO - Processing request of type CallToolRequest


In [ ]:
dac1.print_readable_snapshot()